In [59]:
def inverseMod(a, mod):
    while(a < 0):
        a = a + mod
    
    x1 = 1; x2 = 0; x3 = mod
    y1 = 0; y2 = 1; y3 = a
    
    q = int(x3 / y3)
    t1 = x1 - q*y1
    t2 = x2 - q*y2
    t3 = x3 - (q*y3)
    
    while(y3 != 1):
        x1 = y1; x2 = y2; x3 = y3
        y1 = t1; y2 = t2; y3 = t3
        
        q = int(x3 / y3)
        t1 = x1 - q*y1
        t2 = x2 - q*y2
        t3 = x3 - (q*y3)
    
    while(y2 < 0):
        y2 = y2 + mod
    
    return y2

In [60]:
class Curve:
    def __init__(self, a, b, p):
        # curve configuration
        # y^2 = x^3 + a*x + b
        self.a = a
        self.b = b
        self.p = p
        
defaultCurve = Curve(a=0, b=7, p=199)

In [76]:
class Point:
    def __init__(self, x, y, curve=defaultCurve):
        xp = x % curve.p
        yp = y % curve.p
        
        while(xp < 0):
            xp = xp + curve.p

        while(yp < 0):
            yp = yp + curve.p
        
        self.x = xp
        self.y = yp
        self.curve = curve
        
    def copy(self):
        return Point(self.x, self.y, self.curve)
    
    def __neg__(self):
        return Point(self.x, -self.y, self.curve)
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y and self.curve == other.curve
        
    def __add__(self, other):
        if other.curve != self.curve:
            raise Exception("Can not add with different curves")
        
        if self == other:
            beta = (3 * self.x * self.x + self.curve.a) * inverseMod(2 * self.y, self.curve.p)
        else:
            beta = (other.y - self.y) * inverseMod(other.x - self.x, self.curve.p)

        xnew = beta * beta - self.x - other.x
        ynew = beta * (self.x - xnew) - self.y

        return Point(xnew, ynew, self.curve)
    
    def __sub__(self, other):
        return self + (-other)
    
    def __mul__(self, k):
        temp = self.copy()
        kAsBinary = bin(k)
        kAsBinary = kAsBinary[2:len(kAsBinary)]

        for i in range(1, len(kAsBinary)):
            temp = temp + temp
            if kAsBinary[i: i+1] == '1':
                temp = temp + self

        return temp
    
    def __rmul__(self, k):
        return self * k

g = Point(x=2, y=24)

## ECDH

In [77]:
# Alice gera publica
alicePrivate = 2010000000000017
alicePublic = alicePrivate * g
print(f'{ alicePublic.x = }, { alicePublic.y = }')

 alicePublic.x = 18,  alicePublic.y = 119


In [78]:
# Bob gera publica
bobPrivate = 2010000000000061
bobPublic = bobPrivate * g
print(f'{ bobPublic.x = }, { bobPublic.y = }')

 bobPublic.x = 186,  bobPublic.y = 19


In [79]:
# Alice computa shared a partir de publica de Bob
aliceShared = alicePrivate * bobPublic
print(f'{ aliceShared.x = }, { aliceShared.y = }')

 aliceShared.x = 68,  aliceShared.y = 88


In [80]:
# Bob computa shared a partir de publica de Alice
bobShared = bobPrivate * alicePublic
print(f'{ bobShared.x = }, { bobShared.y = }')

 bobShared.x = 68,  bobShared.y = 88


## Proposed

### Registration (gerar chaves publicas)

In [81]:
# Alice gera publica
alicePrivate = 2010000000000017
alicePublic = alicePrivate * g
print(f'{ alicePublic.x = }, { alicePublic.y = }')

 alicePublic.x = 18,  alicePublic.y = 119


In [82]:
# Bob gera publica
bobPrivate = 2010000000000061
bobPublic = bobPrivate * g
print(f'{ bobPublic.x = }, { bobPublic.y = }')

 bobPublic.x = 186,  bobPublic.y = 19


### Computacao (segredo da sessao e envio)

In [83]:
aliceSessionSecret = 1982639182
toBob = ((alicePrivate + aliceSessionSecret) % defaultCurve.p) * bobPublic
print(f'{ toBob.x = }, { toBob.y = }')

 toBob.x = 103,  toBob.y = 136


In [84]:
bobSessionSecret = 7189234751
toAlice = ((bobPrivate + bobSessionSecret) % defaultCurve.p) * alicePublic
print(f'{ toAlice.x = }, { toAlice.y = }')

 toAlice.x = 162,  toAlice.y = 62


### Computacao chave sessao

In [85]:
aliceShared = inverseMod(alicePrivate, defaultCurve.p) * toAlice - alicePublic
print(f'{ aliceShared.x = }, { aliceShared.y = }')

 aliceShared.x = 158,  aliceShared.y = 125


In [86]:
bobShared = inverseMod(bobPrivate, defaultCurve.p) * toBob - bobPublic
print(f'{ bobShared.x = }, { bobShared.y = }')

 bobShared.x = 15,  bobShared.y = 180
